In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
import sendgrid
import os
from sendgrid import To, Mail, Email, Content
import asyncio

In [2]:
load_dotenv(override=True)

True

In [5]:
def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))
    from_email = Email("vimalpillai1234@gmail.com")
    to_email = To("vimalpillai47@gmail.com")
    content = Content("text/plain","This is a important test email")
    mail = Mail(from_email,to_email,"Test email",content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

202


In [6]:
instructions1 = """You are a sales agent working for ComplAI, 
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. 
You write professional, serious cold emails."""

instructions2 = """You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."""

instructions3 = """You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."""

In [8]:
sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model="gpt-4.1-mini"
)

sales_agent2 = Agent(
    name="Engaging Sales Agent",
    instructions=instructions2,
    model="gpt-4.1-mini"
)

sales_agent3 = Agent(
    name="Busy Sales Agent",
    instructions=instructions3,
    model="gpt-4.1-mini"
)

In [10]:
result = Runner.run_streamed(sales_agent1, "Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta,end="",flush=True)

Subject: Simplify Your SOC 2 Compliance with AI-Powered Automation

Hi [First Name],

Ensuring SOC 2 compliance can be a complex and time-consuming process—especially when preparing for audits. At ComplAI, we’ve developed an AI-powered SaaS tool designed to streamline compliance management and audit readiness, saving your team valuable time and reducing risk.

Our platform helps you:  
- Automate evidence collection and documentation  
- Continuously monitor compliance controls  
- Generate audit-ready reports effortlessly

If improving your SOC 2 compliance process is a priority this quarter, I’d love to schedule a brief call to discuss how ComplAI can support your team.

Would you be open to a 15-minute conversation next week?

Best regards,  
[Your Full Name]  
Sales Representative, ComplAI  
[Your Email] | [Your Phone Number]  
[ComplAI Website]

In [ ]:
message = "Write a cold sales email"
with trace